## Dicts and sets
Dicts and sets use hashtables under the hood

### What is hashable?
An object is hashable if it has a hash which never changes during the object lifetime. The hashable object must implement `__hash()__` and `__eq()__`. <br>
Hashable objects which compare equal must have the same hash. <br><br>

Atomic immutable types are hashable:
- str
- bytes
- numeric types
A `frozen set` is hashable since its elements are hashable by definition. <br>
A tuple is hashable only if its elements are all hashable.

### Dicts
Keys of dict must be hashable <br>

In [2]:
tt = (1, 3, (3, 4))
print(hash(tt))
tl = (1, 3, [3, 4])
try:
    hash(tl)
except TypeError as e:
    print(f"Error: {e}")
tf = (1, 3, frozenset([2, 4]))
print(hash(tf))

7087858078147907433
Error: unhashable type: 'list'
-3125688996852041180


In [4]:
# Building a dict
a = dict(one=1, two=2, three=3)
b = {"one": 1, "two": 2, "three": 3}
c = dict(zip(["one", "two", "three"], [1, 2, 3]))
d = dict([("two", 2), ("one", 1), ("three", 3)])
e = dict({"three": 3, "one": 1, "two": 2})
a == b == c == d == e

True

#### dict comprehensions

In [6]:
DIAL_CODES = [
    (86, "China"),
    (91, "India"),
    (1, "United States"),
    (62, "Indonesia"),
    (55, "Brazil"),
]
country_code = {country: code for code, country in DIAL_CODES}
country_code

{'China': 86, 'India': 91, 'United States': 1, 'Indonesia': 62, 'Brazil': 55}

#### Handling missing keys

In [10]:
# Using setdefault method

import re

WORD_RE = re.compile(r"\w+")

index = {}

fp = """
    Ciao provo a scrivere qualcosa
    per vedere se
    funziona anche se
"""

for line_no, line in enumerate(fp, 1):
    for match in WORD_RE.finditer(line):
        word = match.group()
        column_no = match.start() + 1
        location = (line_no, column_no)
        index.setdefault(word, []).append(location)

print(index)

{'C': [(6, 1)], 'i': [(7, 1), (22, 1), (63, 1)], 'a': [(8, 1), (17, 1), (30, 1), (35, 1), (66, 1), (68, 1)], 'o': [(9, 1), (13, 1), (15, 1), (33, 1), (64, 1)], 'p': [(11, 1), (41, 1)], 'r': [(12, 1), (21, 1), (25, 1), (43, 1), (49, 1)], 'v': [(14, 1), (23, 1), (45, 1)], 's': [(19, 1), (34, 1), (52, 1), (74, 1)], 'c': [(20, 1), (32, 1), (70, 1)], 'e': [(24, 1), (26, 1), (42, 1), (46, 1), (48, 1), (50, 1), (53, 1), (72, 1), (75, 1)], 'q': [(28, 1)], 'u': [(29, 1), (60, 1)], 'l': [(31, 1)], 'd': [(47, 1)], 'f': [(59, 1)], 'n': [(61, 1), (65, 1), (69, 1)], 'z': [(62, 1)], 'h': [(71, 1)]}


In [12]:
# Using collections.defaultdict
# Default dicts are configured to create items on demand whenever a missing key is searched

import re
import collections

WORD_RE = re.compile(r"\w+")

index = collections.defaultdict(list)

fp = """
    Ciao provo a scrivere qualcosa
    per vedere se
    funziona anche se
"""

for line_no, line in enumerate(fp, 1):
    for match in WORD_RE.finditer(line):
        word = match.group()
        column_no = match.start() + 1
        location = (line_no, column_no)
        index[word].append(location)

for word in sorted(index, key=str.upper):
    print(word, index[word])

a [(8, 1), (17, 1), (30, 1), (35, 1), (66, 1), (68, 1)]
C [(6, 1)]
c [(20, 1), (32, 1), (70, 1)]
d [(47, 1)]
e [(24, 1), (26, 1), (42, 1), (46, 1), (48, 1), (50, 1), (53, 1), (72, 1), (75, 1)]
f [(59, 1)]
h [(71, 1)]
i [(7, 1), (22, 1), (63, 1)]
l [(31, 1)]
n [(61, 1), (65, 1), (69, 1)]
o [(9, 1), (13, 1), (15, 1), (33, 1), (64, 1)]
p [(11, 1), (41, 1)]
q [(28, 1)]
r [(12, 1), (21, 1), (25, 1), (43, 1), (49, 1)]
s [(19, 1), (34, 1), (52, 1), (74, 1)]
u [(29, 1), (60, 1)]
v [(14, 1), (23, 1), (45, 1)]
z [(62, 1)]


The `__missing__` method can be used as well, the `__getitem__` looks for it when it can not find a key. <br>
It is better to subclass `collections.UserDict` instead of dict.

#### Variations of dict

Counter

In [2]:
# collections.Counter
import collections

ct = collections.Counter("abracadabra")
print(ct)
ct.update("aaaazzz")
ct.most_common(2)

Counter({'a': 5, 'b': 2, 'r': 2, 'c': 1, 'd': 1})


[('a', 9), ('z', 3)]

#### UserDict
collections.UserDict <br>
It works as a standard dict but it is designed to be subclassed. <br>
This is because the builtin dict has some shortcuts in the implementation.

In [ ]:
# Example: StrKeyDict stores all non-string keys as str
import collections


class StrKeyDict(collections.UserDict):

    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]

    def __contains__(self, key):
        return str(key) in self.data

    def __setitem__(self, key, item):
        self.data[str(key)] = item

### Immutable Mappings

In [4]:
# Example: read only mapping MappingProxyType

from types import MappingProxyType

d = {1: "A"}
d_proxy = MappingProxyType(d)
print(d_proxy)
d[1] = "Z"
print(d_proxy)
d_proxy[2] = "x"

{1: 'A'}
{1: 'Z'}


TypeError: 'mappingproxy' object does not support item assignment

## Sets
A set is a collection of unique objects. <br>
Set elements must be hashable. <br>
The `set` type is not hashable, the `frozenset` type is hashable instead.

In [1]:
# Example: remove duplication
l = ["spam", "spam", "eggs"]
print(set(l))
list(set(l))

{'spam', 'eggs'}


['spam', 'eggs']

Sets supports infix operators:
- a | b --> union
- a & b --> intersection
- a - b --> difference

In [2]:
# Empty set: set()

s = {1}
print(type(s))
print(s)
s.pop()
print(s)

<class 'set'>
{1}
set()


In [3]:
# Literal syntax is faster
from dis import dis

dis("{1}")
dis("set([1])")

  0           RESUME                   0

  1           LOAD_CONST               0 (1)
              BUILD_SET                1
              RETURN_VALUE
  0           RESUME                   0

  1           LOAD_NAME                0 (set)
              PUSH_NULL
              LOAD_CONST               0 (1)
              BUILD_LIST               1
              CALL                     1
              RETURN_VALUE


In [4]:
# Frozenset syntax
frozenset(range(10))

frozenset({0, 1, 2, 3, 4, 5, 6, 7, 8, 9})

In [5]:
# Set Comprehensions

from unicodedata import name

{chr(i) for i in range(32, 256) if "SIGN" in name(chr(i), "")}

{'#',
 '$',
 '%',
 '+',
 '<',
 '=',
 '>',
 '¢',
 '£',
 '¤',
 '¥',
 '§',
 '©',
 '¬',
 '®',
 '°',
 '±',
 'µ',
 '¶',
 '×',
 '÷'}

### Set operations
Check fluent python book

## Dict and set under the hood
Check fluent python

### Hash tables in dictionaries
Check fluent python

In [1]:
# Key ordering depends on insertion order

DIAL_CODES = [(86, "China"), (91, "India"), (1, "United States")]

d1 = dict(DIAL_CODES)
print('d1:', d1.keys())
d2 = dict(sorted(DIAL_CODES))
print('d2:', d2.keys())
d3 = dict(sorted(DIAL_CODES, key=lambda x:x[1]))
print('d3:', d3.keys())
assert d1 == d2 and d2 == d3

d1: dict_keys([86, 91, 1])
d2: dict_keys([1, 86, 91])
d3: dict_keys([86, 91, 1])
